# 13. 텐서플로에서 데이터 적재와 전처리하기

In [1]:
import sklearn
assert sklearn.__version__ >= "0.20"

try:
    # %tensorflow_version은 코랩 명령입니다.
    %tensorflow_version 2.x
    %pip install -q -U tfx
    print("패키지 호환 에러는 무시해도 괜찮습니다.")
except Exception:
    pass

# 텐서플로 ≥2.0 필수
import tensorflow as tf
from tensorflow import keras
assert tf.__version__ >= "2.0"

# 공통 모듈 임포트
import numpy as np
import os

     |████████████████████████████████| 2.4 MB 4.3 MB/s 
     |████████████████████████████████| 17.7 MB 77 kB/s 
     |████████████████████████████████| 19.0 MB 1.2 MB/s 
     |████████████████████████████████| 9.8 MB 47.3 MB/s 
     |████████████████████████████████| 1.7 MB 38.0 MB/s 
     |████████████████████████████████| 40 kB 5.8 MB/s 
     |████████████████████████████████| 407 kB 46.8 MB/s 
     |████████████████████████████████| 1.6 MB 37.2 MB/s 
     |████████████████████████████████| 1.2 MB 15.4 MB/s 
     |████████████████████████████████| 49 kB 5.7 MB/s 
     |████████████████████████████████| 135 kB 61.0 MB/s 
     |████████████████████████████████| 147 kB 59.3 MB/s 
     |████████████████████████████████| 97 kB 7.3 MB/s 
     |████████████████████████████████| 202 kB 56.5 MB/s 
     |████████████████████████████████| 6.5 MB 12.7 MB/s 
     |████████████████████████████████| 1.4 MB 34.1 MB/s 
     |████████████████████████████████| 1.8 MB 32.2 MB/s 
     |████████████████

# 데이터 API

**Dataset**  
- 연속된 데이터 샘플

In [2]:
# 샘플 데이터 텐서 10개
X = tf.range(10)
dataset = tf.data.Dataset.from_tensor_slices(X) # 메모리에서 전체 데이터셋을 생성
dataset

<TensorSliceDataset shapes: (), types: tf.int32>

In [3]:
# 데이터 셋의 아이템 순화
for item in dataset:
    print(item) # 텐서들의 리스트

tf.Tensor(0, shape=(), dtype=int32)
tf.Tensor(1, shape=(), dtype=int32)
tf.Tensor(2, shape=(), dtype=int32)
tf.Tensor(3, shape=(), dtype=int32)
tf.Tensor(4, shape=(), dtype=int32)
tf.Tensor(5, shape=(), dtype=int32)
tf.Tensor(6, shape=(), dtype=int32)
tf.Tensor(7, shape=(), dtype=int32)
tf.Tensor(8, shape=(), dtype=int32)
tf.Tensor(9, shape=(), dtype=int32)


**연쇄 변환**

In [4]:
# 원본 데이터셋에서 repeat() 메소드를 호출하면 원본 데이터셋의 아이템을 세차례 반복하는 새로운 데이터셋을 반환한다.
# 이때 데이터는 세 번 복사하는 것은 아니다.
# 새로운 데이터셋에서 batch() 메서드를 호출하면 다시 새로운 데이터셋이 만들어진다.
dataset = dataset.repeat(3).batch(7)
for item in dataset:
    print(item)

# batch() 메서드에서 출력된 마지막 배치 크기는 2이다.
# batch() 메서드를 drop_remainder=True로 호출하면 길이가 모자란 마지막 배치를 버리고 모든 배치를 동일한 크기로 맞춘다.

tf.Tensor([0 1 2 3 4 5 6], shape=(7,), dtype=int32)
tf.Tensor([7 8 9 0 1 2 3], shape=(7,), dtype=int32)
tf.Tensor([4 5 6 7 8 9 0], shape=(7,), dtype=int32)
tf.Tensor([1 2 3 4 5 6 7], shape=(7,), dtype=int32)
tf.Tensor([8 9], shape=(2,), dtype=int32)


map() : 호출하면 아이템을 변환. 각 아이템에 변환 적용  
apply() : 데이터셋 전체에 변환을 적용  
filter() : 데이터셋을 필터링할 수 있다.  
take() : 데이터셋에 있는 몇 개의 아이템만 보고 싶을 때 사용

In [5]:
# 모든 아이템에 2를 곱하여 새로운 데이터셋 생성
dataset = dataset.map(lambda x: x * 2)
for item in dataset:
    print(item)

tf.Tensor([ 0  2  4  6  8 10 12], shape=(7,), dtype=int32)
tf.Tensor([14 16 18  0  2  4  6], shape=(7,), dtype=int32)
tf.Tensor([ 8 10 12 14 16 18  0], shape=(7,), dtype=int32)
tf.Tensor([ 2  4  6  8 10 12 14], shape=(7,), dtype=int32)
tf.Tensor([16 18], shape=(2,), dtype=int32)


In [6]:
# dataset = dataset.apply(tf.data.experimental.unbatch())

In [7]:
# 새로 만들어진 데이터셋의 각 아이템은 7개의 정수로 이루어진 배치가 아닌 하나의 정수 텐서가 된다.
dataset = dataset.unbatch()

In [8]:
# lambda 안에 만족한 아이템만
dataset = dataset.filter(lambda x: x < 10) 

In [9]:
for item in dataset.take(3):
    print(item)

tf.Tensor(0, shape=(), dtype=int32)
tf.Tensor(2, shape=(), dtype=int32)
tf.Tensor(4, shape=(), dtype=int32)


**데이터 셔플링**  
- Buffer를 채울 때는 순차적으로 반환할 때는 랜덤하게
- Buffer size가 크지 않으면 shuffling 효과각 감소

In [10]:
tf.random.set_seed(42)

dataset = tf.data.Dataset.range(10).repeat(3)
dataset = dataset.shuffle(buffer_size=3, seed=42).batch(7)
for item in dataset:
    print(item)

# 정수 0에서 9까지 세번 반복된 데이터셋을 만든 다음 버퍼 크기 5와 랜덤 시드 42을 사용하여 셔플링하고 배치크기 7로 나누어 출력한다.

tf.Tensor([1 3 0 4 2 5 6], shape=(7,), dtype=int64)
tf.Tensor([8 7 1 0 3 2 5], shape=(7,), dtype=int64)
tf.Tensor([4 6 9 8 9 7 0], shape=(7,), dtype=int64)
tf.Tensor([3 1 4 5 2 8 7], shape=(7,), dtype=int64)
tf.Tensor([6 9], shape=(2,), dtype=int64)


**캘리포니아 주택 데이터셋을 여러 개의 CSV로 나누기**

In [11]:
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

housing = fetch_california_housing()
X_train_full, X_test, y_train_full, y_test = train_test_split(
    housing.data, housing.target.reshape(-1, 1), random_state=42)
X_train, X_valid, y_train, y_valid = train_test_split(
    X_train_full, y_train_full, random_state=42)

scaler = StandardScaler()
scaler.fit(X_train)
X_mean = scaler.mean_
X_std = scaler.scale_

def save_to_multiple_csv_files(data, name_prefix, header=None, n_parts=10):
    housing_dir = os.path.join("datasets", "housing")
    os.makedirs(housing_dir, exist_ok=True)
    path_format = os.path.join(housing_dir, "my_{}_{:02d}.csv")

    filepaths = []
    m = len(data)
    for file_idx, row_indices in enumerate(np.array_split(np.arange(m), n_parts)):
        part_csv = path_format.format(name_prefix, file_idx)
        filepaths.append(part_csv)
        with open(part_csv, "wt", encoding="utf-8") as f:
            if header is not None:
                f.write(header)
                f.write("\n")
            for row_idx in row_indices:
                f.write(",".join([repr(col) for col in data[row_idx]]))
                f.write("\n")
    return filepaths

train_data = np.c_[X_train, y_train]
valid_data = np.c_[X_valid, y_valid]
test_data = np.c_[X_test, y_test]
header_cols = housing.feature_names + ["MedianHouseValue"]
header = ",".join(header_cols)

train_filepaths = save_to_multiple_csv_files(train_data, "train", header, n_parts=20)
valid_filepaths = save_to_multiple_csv_files(valid_data, "valid", header, n_parts=10)
test_filepaths = save_to_multiple_csv_files(test_data, "test", header, n_parts=10)

In [12]:
# 캘리포니아 주택 데이터셋 적재하고 섞은 다음 훈련세트, 검증세트, 테스트 세트로 나누었다고 가정
# 각 세트를 csv 파일 여러개로 나눔
with open(train_filepaths[0]) as f:
    for i in range(5):
        print(f.readline(), end="")

MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude,MedianHouseValue
3.5214,15.0,3.0499445061043287,1.106548279689234,1447.0,1.6059933407325193,37.63,-122.43,1.442
5.3275,5.0,6.490059642147117,0.9910536779324056,3464.0,3.4433399602385686,33.69,-117.39,1.687
3.1,29.0,7.5423728813559325,1.5915254237288134,1328.0,2.2508474576271187,38.44,-122.98,1.621
7.1736,12.0,6.289002557544757,0.9974424552429667,1054.0,2.6956521739130435,33.55,-117.7,2.621


In [13]:
# 훈련 파일 경로를 담은 리스트라고 가정
train_filepaths

['datasets/housing/my_train_00.csv',
 'datasets/housing/my_train_01.csv',
 'datasets/housing/my_train_02.csv',
 'datasets/housing/my_train_03.csv',
 'datasets/housing/my_train_04.csv',
 'datasets/housing/my_train_05.csv',
 'datasets/housing/my_train_06.csv',
 'datasets/housing/my_train_07.csv',
 'datasets/housing/my_train_08.csv',
 'datasets/housing/my_train_09.csv',
 'datasets/housing/my_train_10.csv',
 'datasets/housing/my_train_11.csv',
 'datasets/housing/my_train_12.csv',
 'datasets/housing/my_train_13.csv',
 'datasets/housing/my_train_14.csv',
 'datasets/housing/my_train_15.csv',
 'datasets/housing/my_train_16.csv',
 'datasets/housing/my_train_17.csv',
 'datasets/housing/my_train_18.csv',
 'datasets/housing/my_train_19.csv']

In [14]:
# list_files() : 파일 경로를 섞은 데이터셋 반환
# 원하지 않다면 shuffle=False
filepath_dataset = tf.data.Dataset.list_files(train_filepaths, seed=42)

for filepath in filepath_dataset:
    print(filepath)

tf.Tensor(b'datasets/housing/my_train_15.csv', shape=(), dtype=string)
tf.Tensor(b'datasets/housing/my_train_08.csv', shape=(), dtype=string)
tf.Tensor(b'datasets/housing/my_train_03.csv', shape=(), dtype=string)
tf.Tensor(b'datasets/housing/my_train_01.csv', shape=(), dtype=string)
tf.Tensor(b'datasets/housing/my_train_10.csv', shape=(), dtype=string)
tf.Tensor(b'datasets/housing/my_train_05.csv', shape=(), dtype=string)
tf.Tensor(b'datasets/housing/my_train_19.csv', shape=(), dtype=string)
tf.Tensor(b'datasets/housing/my_train_16.csv', shape=(), dtype=string)
tf.Tensor(b'datasets/housing/my_train_02.csv', shape=(), dtype=string)
tf.Tensor(b'datasets/housing/my_train_09.csv', shape=(), dtype=string)
tf.Tensor(b'datasets/housing/my_train_00.csv', shape=(), dtype=string)
tf.Tensor(b'datasets/housing/my_train_07.csv', shape=(), dtype=string)
tf.Tensor(b'datasets/housing/my_train_12.csv', shape=(), dtype=string)
tf.Tensor(b'datasets/housing/my_train_04.csv', shape=(), dtype=string)
tf.Ten

In [15]:
# interleave() : 한 번에 다섯개의 파일을 한 줄씩 번갈아 읽는다.
# 각 파일의 첫번째 줄은 열 이름이다. -> skip() 메서드 사용하여 건너뛴다.
n_readers = 5
dataset = filepath_dataset.interleave(
    lambda filepath: tf.data.TextLineDataset(filepath).skip(1),
    cycle_length=n_readers)

In [16]:
# csv 파일의 첫번째 행에 행당하며 순서는 랜덤하다.
for line in dataset.take(5):
    print(line.numpy())

b'4.6477,38.0,5.03728813559322,0.911864406779661,745.0,2.5254237288135593,32.64,-117.07,1.504'
b'8.72,44.0,6.163179916317992,1.0460251046025104,668.0,2.794979079497908,34.2,-118.18,4.159'
b'3.8456,35.0,5.461346633416459,0.9576059850374065,1154.0,2.8778054862842892,37.96,-122.05,1.598'
b'3.3456,37.0,4.514084507042254,0.9084507042253521,458.0,3.2253521126760565,36.67,-121.7,2.526'
b'3.6875,44.0,4.524475524475524,0.993006993006993,457.0,3.195804195804196,34.04,-118.15,1.625'


**데이터 전처리**

In [17]:
# 전처리를 수행하기 위한 함수 정의
n_inputs = 8 # X_train.shape[-1]

@tf.function
# preprocess() 함수는 csv 한 라인을 받아 파싱한다.
def preprocess(line):
    defs = [0.] * n_inputs + [tf.constant([], dtype=tf.float32)]
    # decode_csv() 함수는 (열마다 한 개씩) 스칼라 텐서의 리스트를 반환한다.
    fields = tf.io.decode_csv(line, record_defaults=defs)
    x = tf.stack(fields[:-1])
    y = tf.stack(fields[-1:])
    # 입력 특성에서 평균을 빼고 표준편차로 나누어 스케일을 조정한다.
    return (x - X_mean) / X_std, y

In [18]:
# 전처리 함수 테스트
preprocess(b'4.2083,44.0,5.3232,0.9171,846.0,2.3370,37.47,-122.2,2.782')

(<tf.Tensor: shape=(8,), dtype=float32, numpy=
 array([ 0.16579157,  1.216324  , -0.05204565, -0.39215982, -0.5277444 ,
        -0.2633488 ,  0.8543046 , -1.3072058 ], dtype=float32)>,
 <tf.Tensor: shape=(1,), dtype=float32, numpy=array([2.782], dtype=float32)>)

**프리페치**
- 프리페치로 CPU, GPU를 동시에 사용
- GPU가 한 배치를 처리할 때 CPU가 그 다음 배치를 준비한다.

**데이터 적재와 전처리를 합치기**

In [19]:
# 지금까지 언급했던 모든 것을 하나의 헬퍼함수로 만든다.
# CSV 파일에서 캘리포니아 주택 데이터셋을 효율적으로 적재하고 전처리, 셔플링, 반복, 배치를 적용한 데이터셋을 만들어 반환한다.
def csv_reader_dataset(filepaths, repeat=1, n_readers=5,
                       n_read_threads=None, shuffle_buffer_size=10000,
                       n_parse_threads=5, batch_size=32):
    dataset = tf.data.Dataset.list_files(filepaths).repeat(repeat)
    dataset = dataset.interleave(
        lambda filepath: tf.data.TextLineDataset(filepath).skip(1),
        cycle_length=n_readers, num_parallel_calls=n_read_threads)
    dataset = dataset.shuffle(shuffle_buffer_size)
    dataset = dataset.map(preprocess, num_parallel_calls=n_parse_threads)
    dataset = dataset.batch(batch_size)
    return dataset.prefetch(1)

**tf.keras와 데이터셋 사용하기**

In [20]:
# csv_reader_dataset() 함수로 훈련 세트로 사용할 데이터셋을 만들 수 있다.
train_set = csv_reader_dataset(train_filepaths, repeat=None)
valid_set = csv_reader_dataset(valid_filepaths)
test_set = csv_reader_dataset(test_filepaths)

In [21]:
keras.backend.clear_session()
np.random.seed(42)
tf.random.set_seed(42)

model = keras.models.Sequential([
    keras.layers.Dense(30, activation="relu", input_shape=X_train.shape[1:]),
    keras.layers.Dense(1),
])

In [22]:
model.compile(loss="mse", optimizer=keras.optimizers.SGD(learning_rate=1e-3))

In [23]:
batch_size = 32
# fit() 메서드에 훈련 데이터셋과 검증 데이터셋 전달
model.fit(train_set, steps_per_epoch=len(X_train) // batch_size, epochs=10,
          validation_data=valid_set)

Epoch 1/10
362/362 [==============================] - 2s 3ms/step - loss: 1.4500 - val_loss: 21.6364
Epoch 2/10
362/362 [==============================] - 1s 3ms/step - loss: 0.9054 - val_loss: 0.7100
Epoch 3/10
362/362 [==============================] - 1s 3ms/step - loss: 0.6092 - val_loss: 0.6093
Epoch 4/10
362/362 [==============================] - 1s 3ms/step - loss: 0.6023 - val_loss: 0.5711
Epoch 5/10
362/362 [==============================] - 1s 3ms/step - loss: 0.5790 - val_loss: 0.6348
Epoch 6/10
362/362 [==============================] - 1s 2ms/step - loss: 0.5521 - val_loss: 0.6665
Epoch 7/10
362/362 [==============================] - 1s 3ms/step - loss: 0.5219 - val_loss: 0.5160
Epoch 8/10
362/362 [==============================] - 1s 3ms/step - loss: 0.5318 - val_loss: 0.4933
Epoch 9/10
362/362 [==============================] - 1s 3ms/step - loss: 0.4981 - val_loss: 0.4706
Epoch 10/10
362/362 [==============================] - 1s 3ms/step - loss: 0.4712 - val_loss: 0.467

In [24]:
# new_set은 레이블을 가지고 있지 않다.
model.evaluate(test_set, steps=len(X_test) // batch_size)
new_set = test_set.map(lambda X, y: X) # 새로운 샘플이 3개 있다고 가정
model.predict(new_set)

161/161 [==============================] - 0s 2ms/step - loss: 0.4788


array([[1.1898293],
       [2.2555187],
       [1.88677  ],
       ...,
       [1.4365605],
       [1.1184465],
       [1.5204434]], dtype=float32)

# 입력 특성 전처리

**layer 정의**

In [25]:
# 사용자 정의 층
class Standardization(keras.layers.Layer):
    def adapt(self, data_sample):
        self.means_ = np.mean(data_sample, axis=0, keepdims=True)
        self.stds_ = np.std(data_sample, axis=0, keepdims=True)
    def call(self, inputs):
        return (inputs - self.means_) / (self.stds_ + keras.backend.epsilon())

# Standardization 층을 모델에 추가하기 전에 데이터 샘플과 함께 adapt() 메서드를 호출
# 각 특성에 대해 적절한 평균과 표준편차를 사용가능

**원-핫 벡터를 사용해 범주형 특성 인코딩하기**  
- 범주형의 특성을 표현할 때 사용

In [26]:
# 어학 사전 정의
vocab = ['<1H OCEAN', 'INLAND', 'ISLAND', 'NEAR BAY', 'NEAR OCEAN']
indices = tf.range(len(vocab), dtype=tf.int64)
table_init = tf.lookup.KeyValueTensorInitializer(vocab, indices)

# 초기화 객체와 oov 버킷을 지정하여 룩업 테이블을 만든다.
# 어학 사전에 없는 범주를 찾으면 룩업 테이블이 계산한 이 범주의 해시값을 이용하여 oov 버킷 중 하나를 할당한다.
# 훈련 도중 발견되는 알려지지 않은 범주들이 많을수록 더 많은 oov 버킷을 사용해야 한다.
num_oov_buckets = 2
table = tf.lookup.StaticVocabularyTable(table_init, num_oov_buckets)

In [27]:
# 룩업 테이블을 사용해 몇 개의 범주 특성을 원-핫 벡터로 인코딩
categories = tf.constant(['NEAR BAY', 'NEAR BAY', 'INLAND', 'INLAND'])
cat_indices = table.lookup(categories)
cat_indices

<tf.Tensor: shape=(4,), dtype=int64, numpy=array([3, 3, 1, 1])>

In [28]:
cat_one_hot = tf.one_hot(cat_indices, depth=len(vocab) + num_oov_buckets)
cat_one_hot

<tf.Tensor: shape=(4, 7), dtype=float32, numpy=
array([[0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0.]], dtype=float32)>

**임베딩을 사용해 범주형 특성 인코딩**  
- 랜덤하게 초기화했다가 훈련하여 얻는 밀집 벡터 표현
- 표현 학습  
  - 비슷한 속성의 category가 공간적으로 가깝게 표현됨
  - 의미 있는 축의 속성으로 표현됨

In [29]:
# 각 번주의 임베딩을 담은 입베딩 행렬 랜덤하게 초기화
embedding_dim = 2
# [len(vocab) + num_oov_buckets, embedding_dim] : 범주와 oov 버킷마다 하나의 행이 있고 임베딩 차원마다 하나의 열을 가진다.
embed_init = tf.random.uniform([len(vocab) + num_oov_buckets, embedding_dim])
embedding_matrix = tf.Variable(embed_init)

In [30]:
# 랜덤한 6X2 행렬
embedding_matrix

<tf.Variable 'Variable:0' shape=(7, 2) dtype=float32, numpy=
array([[0.7413678 , 0.62854624],
       [0.01738465, 0.3431449 ],
       [0.51063764, 0.3777541 ],
       [0.07321596, 0.02137029],
       [0.2871771 , 0.4710616 ],
       [0.6936141 , 0.07321334],
       [0.93251204, 0.20843053]], dtype=float32)>

In [31]:
# 범주 특성 인코딩
categories = tf.constant(['NEAR BAY', 'NEAR BAY', 'INLAND', 'INLAND'])
cat_indices = table.lookup(categories)
cat_indices

<tf.Tensor: shape=(4,), dtype=int64, numpy=array([3, 3, 1, 1])>

In [32]:
# tf.nn.embedding_lookup() : 임베딩 행렬에서 주어진 인덱스에 해당하는 행을 찾는다.
tf.nn.embedding_lookup(embedding_matrix, cat_indices)

<tf.Tensor: shape=(4, 2), dtype=float32, numpy=
array([[0.07321596, 0.02137029],
       [0.07321596, 0.02137029],
       [0.01738465, 0.3431449 ],
       [0.01738465, 0.3431449 ]], dtype=float32)>

In [33]:
# 케라스는 임베딩 행렬 처리 층을 제공
# 이 층이 생성될 때 임베딩 행렬을 랜덤하게 초기화하고 어떤 범주 인덱스로 호출될 대 임베딩 행렬에 있는 그 인덱스의 행을 반환한다.
embedding = keras.layers.Embedding(input_dim=len(vocab) + num_oov_buckets, output_dim=embedding_dim)
embedding(cat_indices)

<tf.Tensor: shape=(4, 2), dtype=float32, numpy=
array([[ 0.01289039,  0.0191061 ],
       [ 0.01289039,  0.0191061 ],
       [ 0.00692506, -0.00518861],
       [ 0.00692506, -0.00518861]], dtype=float32)>

In [34]:
# 범주형 특성을 처리하고 각 범주, oov 버킷마다 임베딩 학습을 하는 케라스 모델
regular_inputs = keras.layers.Input(shape=[8])
categories = keras.layers.Input(shape=[], dtype=tf.string)
cat_indices = keras.layers.Lambda(lambda cats : table.lookup(cats))(categories)
cat_embed = keras.layers.Embedding(input_dim=6, output_dim=2)(cat_indices)
encoded_inputs = keras.layers.concatenate([regular_inputs, cat_embed])
outputs = keras.layers.Dense(1)(encoded_inputs)
# 샘플마다 8개의 특성을 담은 입력과 하나의 범주형 입력을 받는다.
model = keras.models.Model(inputs=[regular_inputs, categories], outputs=[outputs])

**Keras 전처리 층**  
1. tf.keras.layers.experimental.preprocessing
2. keras.layers.Normalization : 특성 표준화를 수행
3. keras.layers.TextVectorization : 입력에 있는 각 단어를 어휘 사전에 있는 인덱서를 인코딩
4. keras.layers.Discretization : 연속적인 데이터를 몇 개의 구간으로 나누고 각 구간을 원-핫 벡터로 인코딩
5. keras.layers.PreprocessingStage : 여러 전처리 층을 연결

# TFRecord 포맷
- 크기가 다른 연속인 이진 레코드를 저장하기 위한 구조
- 이미지나 오디오 같은 대규모의 복잡한 데이터 구조를 효율적으로 처리

In [35]:
# tf.io.TFRecordWriter -> TFRecord를 손쉽게 만들 수 있다.
with tf.io.TFRecordWriter("my_data.tfrecord") as f:
    f.write(b"This is the first record")
    f.write(b"And this is the second record")

In [36]:
filepaths = ["my_data.tfrecord"]
# tf.data.TFRecordDataset() : 하나 이상의 TFRecord를 읽을 수 있다.
dataset = tf.data.TFRecordDataset(filepaths)
for item in dataset:
    print(item)

tf.Tensor(b'This is the first record', shape=(), dtype=string)
tf.Tensor(b'And this is the second record', shape=(), dtype=string)


In [37]:
# 압축된 TFRecord 파일 생성
options = tf.io.TFRecordOptions(compression_type="GZIP")
with tf.io.TFRecordWriter("my_compressed.tfrecord", options) as f:
    f.write(b"This is the first record")
    f.write(b"And this is the second record")

In [38]:
# 압축 읽어오기 -> 압축 형식을 지정
dataset = tf.data.TFRecordDataset(["my_compressed.tfrecord"],
                                  compression_type="GZIP")
for item in dataset:
    print(item)

tf.Tensor(b'This is the first record', shape=(), dtype=string)
tf.Tensor(b'And this is the second record', shape=(), dtype=string)


**프로토콜 버퍼**  
- TFRecord가 데이터를 다루는 포맷

In [39]:
# Person 프로토콜 버퍼로 생성된 클래스 사용
# 직렬화된 person 객체를 TFRecord 파일로 저장한 다음 읽고 파싱할 수 있다.
%%writefile person.proto
syntax = "proto3";
message Person {
  string name = 1;
  int32 id = 2;
  repeated string email = 3;
}

Writing person.proto


In [40]:
!protoc person.proto --python_out=. --descriptor_set_out=person.desc --include_imports

In [41]:
!ls person*

person.desc  person_pb2.py  person.proto


In [42]:
from person_pb2 import Person

person = Person(name="Al", id=123, email=["a@b.com"])  # Person 생성
print(person)  # Person 출력

name: "Al"
id: 123
email: "a@b.com"



In [43]:
person.name  # 필드 읽기

'Al'

In [44]:
person.name = "Alice"  # 필드 수정

In [45]:
person.email[0]  # 배열처럼 사용할 수 있는 반복 필드

'a@b.com'

In [46]:
person.email.append("c@d.com")  # 이메일 추가

In [47]:
s = person.SerializeToString()  # 바이트 문자열로 직렬화
s

b'\n\x05Alice\x10{\x1a\x07a@b.com\x1a\x07c@d.com'

In [48]:
person2 = Person()  # 새로운 Person 생성
person2.ParseFromString(s)  # 바이트 문자열 파싱 (27 바이트)

27

In [49]:
person == person2  # 동일

True

In [50]:
# tf.train.Example 객체 생성, TFRecord 파일에 저장
from tensorflow.train import BytesList, FloatList, Int64List
from tensorflow.train import Feature, Features, Example

person_example = Example(
    features=Features(
        feature={
            "name": Feature(bytes_list=BytesList(value=[b"Alice"])),
            "id": Feature(int64_list=Int64List(value=[123])),
            "emails": Feature(bytes_list=BytesList(value=[b"a@b.com", b"c@d.com"]))
        }))

with tf.io.TFRecordWriter("my_contacts.tfrecord") as f:
    f.write(person_example.SerializeToString())

In [51]:
# 설명 딕셔너리 정의
feature_description = {
    "name": tf.io.FixedLenFeature([], tf.string, default_value=""),
    "id": tf.io.FixedLenFeature([], tf.int64, default_value=0),
    "emails": tf.io.VarLenFeature(tf.string),
}
# TFRecordDataset을 순회 -> 데이터셋에 포함된 직렬화 Example 프로토콜 버퍼 파싱
for serialized_example in tf.data.TFRecordDataset(["my_contacts.tfrecord"]):
    parsed_example = tf.io.parse_single_example(serialized_example,
                                                feature_description)

# TFDS(텐서플로 데이터셋)
- 널리 사용하는 데이터셋 제공

In [52]:
import tensorflow_datasets as tfds

# MNIST 데이터셋 다운
datasets = tfds.load(name="mnist")
mnist_train, mnist_test = datasets["train"], datasets["test"]

local data directory. If you'd instead prefer to read directly from our public
GCS bucket (recommended if you're running on GCP), you can instead pass
`try_gcs=True` to `tfds.load` or set `data_dir=gs://tfds-data/datasets`.



Dl Completed...:   0%|          | 0/4 [00:00<?, ? file/s]


Dataset mnist downloaded and prepared to /root/tensorflow_datasets/mnist/3.0.1. Subsequent calls will reuse this data.


In [53]:
# 변환을 적용한 다음 모델을 훈련하기 위한 준비를 마침
mnist_train = mnist_train.repeat(5).batch(32)
mnist_train = mnist_train.map(lambda items: (items["image"], items["label"]))
mnist_train = mnist_train.prefetch(1)

In [54]:
datasets = tfds.load(name="mnist", batch_size=32, as_supervised=True)
mnist_train = datasets["train"].repeat().prefetch(1)
model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28, 1]),
    keras.layers.Lambda(lambda images: tf.cast(images, tf.float32)),
    keras.layers.Dense(10, activation="softmax")])
model.compile(loss="sparse_categorical_crossentropy",
              optimizer=keras.optimizers.SGD(learning_rate=1e-3),
              metrics=["accuracy"])
model.fit(mnist_train, steps_per_epoch=60000 // 32, epochs=5)

Epoch 1/5
1875/1875 [==============================] - 8s 4ms/step - loss: 31.8888 - accuracy: 0.8426
Epoch 2/5
1875/1875 [==============================] - 2s 1ms/step - loss: 25.6125 - accuracy: 0.8684
Epoch 3/5
1875/1875 [==============================] - 3s 1ms/step - loss: 24.9121 - accuracy: 0.8737
Epoch 4/5
1875/1875 [==============================] - 2s 1ms/step - loss: 24.3117 - accuracy: 0.8766
Epoch 5/5
1875/1875 [==============================] - 3s 1ms/step - loss: 23.9062 - accuracy: 0.8774
